In [1]:
import sys
sys.path.append('..')

import torch
from torch.nn import BCELoss, BCEWithLogitsLoss
from torch.optim import Adam
from torchvision.models import resnet18

from lib.models import SimpleClassifier, ResNet
from lib.trainer import Trainer
from lib.utils import train_parse_args, get_data_loaders
from lib.constants import HYPERPARAMETERS

In [2]:
params = HYPERPARAMETERS
params['batch_size'] = 8
params['n_epochs'] = 100
params['verbose'] = True
params['version'] = 0.2

if not params['disable_cuda'] and torch.cuda.is_available():
    params['device'] = torch.device('cuda:0')
else:
    params['device'] = torch.device('cpu')
    
for p in params:
    print(f"{p} : \t{params[p]}")

batch_size : 	8
checkpoints_dir : 	/home/sergevkim/git/FacesClassification/checkpoints
checkpoint_filename : 	
disable_cuda : 	False
imgs_dir : 	/home/sergevkim/git/FacesClassification/data/CelebaHQ
label : 	Male
labels_filename : 	/home/sergevkim/git/FacesClassification/data/list_attr_celeba.txt
logs_dir : 	/home/sergevkim/git/FacesClassification/logs
n_epochs : 	100
n_imgs : 	30000
verbose : 	True
version : 	0.2
device : 	cuda:0


In [ ]:
loaders = get_data_loaders(
    imgs_dir=params['imgs_dir'],
    labels_filename=params['labels_filename'],
    batch_size=params['batch_size'],
    n_imgs=params['n_imgs'])

model = ResNet()
model.cuda()

optimizer = Adam(model.parameters(), lr=3e-4)
criterion = BCELoss()
trainer = Trainer(params, model, optimizer, criterion)

trainer.run(loaders)

O [[0.38460222]
 [0.29153329]
 [0.32071307]
 [0.34485668]
 [0.38858312]
 [0.31467655]
 [0.27606705]
 [0.22474693]]
L [1. 0. 1. 0. 1. 1. 0. 1.]
0 0.375
O [[0.30641448]
 [0.24936967]
 [0.31057975]
 [0.34541997]
 [0.25742894]
 [0.31891531]
 [0.27103686]
 [0.25948778]]
L [0. 0. 0. 0. 0. 0. 0. 0.]
100 1.0
O [[0.26400033]
 [0.23969446]
 [0.3114551 ]
 [0.26823622]
 [0.28114253]
 [0.27677301]
 [0.23411159]
 [0.35098729]]
L [0. 0. 1. 0. 1. 1. 1. 0.]
200 0.5
O [[0.30927852]
 [0.28217119]
 [0.3194924 ]
 [0.29797339]
 [0.3061164 ]
 [0.33396411]
 [0.34818593]
 [0.26421607]]
L [0. 1. 1. 1. 0. 1. 1. 0.]
300 0.375
! 0.5743333333333334
EPOCH 1
train 1 0 0.6882690730583028
l tensor([0., 0., 0., 1., 1., 1., 1., 0.], device='cuda:0', dtype=torch.float64)
o tensor([[0.3551],
        [0.3901],
        [0.4078],
        [0.3932],
        [0.4049],
        [0.4216],
        [0.4152],
        [0.3743]], device='cuda:0', dtype=torch.float64,
       grad_fn=<CopyBackwards>)
train 1 100 0.8763382557352173
l tenso

In [ ]:
loaders = get_data_loaders(
    imgs_dir=params['imgs_dir'],
    labels_filename=params['labels_filename'],
    batch_size=params['batch_size'],
    n_imgs=params['n_imgs'])

model = ResNet()
model.cuda()

optimizer = Adam(model.parameters(), lr=3e-4)
criterion = BCELoss()
trainer = Trainer(params, model, optimizer, criterion)

trainer.run(loaders)

In [ ]:
from lib.models import get_resnet
from lib.utils import log_grad_norm


loaders = get_data_loaders(
    imgs_dir=params['imgs_dir'],
    labels_filename=params['labels_filename'],
    batch_size=params['batch_size'],
    n_imgs=params['n_imgs'])

model = get_resnet()
#model.fc.register_backward_hook(log_grad_norm)
#model = SimpleClassifier()
model.cuda()

optimizer = Adam(model.parameters(), lr=1e-4)
criterion = BCEWithLogitsLoss()
trainer = Trainer(params, model, optimizer, criterion)

trainer.run(loaders)

In [ ]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from torchvision.transforms import ToTensor


a = np.array(Image.open("data/CelebaHQ/10.png"))
print(type(a))
plt.imshow(a)
a = ToTensor()(a).cuda()
a = a[np.newaxis, ...]
print(a.shape)


print(model(a))

In [9]:
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import normalize
from scipy.linalg import norm

a = np.array([0, 1, 1])
b = np.array([0.1, 0.2, 2.])

#b = normalize(b[..., np.newaxis])
b = np.round(b)
print(b)

accuracy_score(a, b, normalize=True)

[0. 0. 2.]


0.3333333333333333

In [ ]:
model = get_resnet()
print(model)

In [ ]:
import torch 
a = torch.ones(5)
a.requires_grad = True

b = 2*a

b.retain_grad()   # Since b is non-leaf and it's grad will be destroyed otherwise.

c = b.mean()

c.backward()

print(a.grad, b.grad)

# Redo the experiment but with a hook that multiplies b's grad by 2. 
a = torch.ones(5)

a.requires_grad = True

b = 2*a

b.retain_grad()

b.register_hook(lambda x: print('!', x))  
print(123)
b.mean().backward() 
print(456)
print(a.grad, b.grad)